# Geocoding apartment addresses using the geoadmin API

## Libraries and settings

In [6]:
# Libraries
import os
import sqlite3
import requests
import json
import urllib
import fnmatch
import folium
import math
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Importing diesel data

In [7]:
# Read the data to a pandas data frame
df_diesel = pd.read_csv('data/diesel.csv', 
                 sep=',', 
                 encoding='utf-8')

# Show first records
df_diesel

,date,brand,adress,diesel
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.120
1,04.07.2022,Eni,"Rte d'Aigle 18, 1867 Ollon VD",2.450
2,15.06.2022,Tamoil,"Rte du Nant-d'Avril 36, 1214 Vernier",2.440
3,02.06.2022,BP Service Weinland West,"A4 Ausfahrt Andelfingen, 8452 Adlikon",2.439
4,02.06.2022,BP Service Weinland Ost,"A4 Ausfahrt Andelfingen, 8452 Adlikon",2.439
...,...,...,...,...
2600,04.05.2016,Auto Lugano,"Via Cantonale, 6983 Magliaso",1.319
2601,26.12.2020,Lang Energie AG,"Seestr. 13, 8280 Kreuzlingen",1.310
2602,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.310
2603,22.12.2020,Dorfgarage Willi AG,"Rheintalstrasse 59, 5325 Leibstadt",1.300


# Importing bleifrei data

In [8]:
# Read the data to a pandas data frame
df_bleifrei = pd.read_csv("data/bleifrei.csv", 
                 sep=',', 
                 encoding='utf-8')

# Show first records
df_bleifrei

,date,brand,adress,bleifrei
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.14
1,04.07.2022,Tamoil,"Gsteigstr. 174, 3785 Gsteig b. Gstaad",2.23
2,04.07.2022,Agrola,"Kantonsstr. 50, 3930 Eyholz",2.23
3,04.07.2022,Garage Blatter AG,"Ennet Brücke 7, 3930 Visp",2.23
4,11.07.2022,Garage Gasparini,"Veia Granda, 7440 Andeer",2.23
...,...,...,...,...
2599,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.26
2600,22.12.2020,Dorfgarage Willi AG,"Rheintalstrasse 59, 5325 Leibstadt",1.25
2601,24.07.2016,Garage Jaques,"Route de Lausanne, 1260 Nyon",1.25
2602,27.11.2020,Hansruedi Julmi Garage & Tankstelle,"Bielstrasse 85, 2542 Pieterlen",1.25


In [9]:
df = pd.merge(df_diesel, df_bleifrei, on=['date','adress'])

df = df.drop(columns=['brand_y'])
df = df.rename(columns={"brand_x":"brand", "adress":"address", "diesel":"diesel_price", "bleifrei":"bleifrei_price"})
df = df.dropna()
df = df.drop_duplicates()

df

,date,brand,address,diesel_price,bleifrei_price
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.120,1.140
16,28.10.2022,Socar Gotthard,"A2 West, 6467 Schattdorf",2.395,2.035
17,14.08.2022,BP Rheintal,"Autobahn A13, 9470 Buchs SG",2.380,2.200
18,11.07.2022,eni - Castasegna Ovest,"Farzett, 7608 Castasegna",2.380,2.230
19,03.08.2022,BP,"Grüningerstr. 80, 8624 Grüt (Gossau ZH)",2.370,2.220
...,...,...,...,...,...
2444,04.05.2016,Auto Bellinzona Nord Sagl,"Via S. Gottardo 57B, 6500 Bellinzona",1.320,1.330
2445,02.02.2016,Reisch AG,"Hauptstr. 24, 8573 Siegershausen",1.320,1.270
2446,04.05.2016,Auto Lugano,"Via Cantonale, 6983 Magliaso",1.319,1.279
2447,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.310,1.260


### Geocoding multiple apartment addresses using the geoadmin API

In [10]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df['address'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['address']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['address'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']

Geocoding address 2405 out of 2405 : Rheintalstrasse 59, 5325 Leibstadt


In [11]:
counter=0
count=0
count2=0

for i in df['lat']:
    if math.isnan(i):
        count = count + 1
    else:
        count2 = count2 + 1
        
    counter = counter + 1
    
print(count)
print(count2)

647
1758


### Plot addresses on map

In [12]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add lat/lon of addresses
df_sub = df.dropna()
for i in range(0, len(df_sub)):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon']), 
                  popup=df_sub.iloc[i]['address']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m

### Save data to file

In [13]:
df.to_csv('Master/combined_loc.csv', 
           sep=",", 
           encoding='utf-8',
           index=False)

In [19]:
# Get current working directory
print(os.getcwd())

# Create data base
conn = sqlite3.connect('Master/fuel_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

c:\Users\calvi\OneDrive\Dokumente\GitHub\TS


In [20]:
cursor.execute('''CREATE TABLE IF NOT EXISTS fuel_table ( date VARCHAR(50),
                                                                brand VARCHAR(50),
                                                                address VARCHAR(50),
                                                                diesel_price DECIMAL(4,3),
                                                                bleifrei_price DECIMAL(4,3),
                                                                lat DECIMAL(4,3),
                                                                lon DECIMAL(4,3))''')
# Confirm changes to the table
conn.commit()

In [21]:
df.to_sql(name = 'fuel_table',
          con = conn,
          index = False,
          if_exists = 'replace')

2405

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [15]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2022-12-24 16:53:27
Python Version: 3.11.0
-----------------------------------
